In [13]:
import jax
import jax.numpy as jnp

def assign_returns():
    batch_size = 3
    capacity = 5
    gamma = 1
    start_idx = jnp.array([4, 1, 0])
    next_idx = jnp.array([3, 2, 4])
    reward = jnp.stack([jnp.arange(capacity)]*batch_size)
    print(f"reward: {reward}")
    final_value = 10*jnp.ones(batch_size)
    

    def roll_row(row, shift):
        return jnp.roll(row, shift)
    
    shift = capacity - next_idx 
    rewards = jax.vmap(roll_row)(reward, shift)
    print(f"rewards: {rewards}")
    arr = jnp.zeros((batch_size, capacity)).at[:, -1].set(final_value)
    print(f"arr: {arr}")
    mask = jnp.arange(capacity) >= ((start_idx + shift)%capacity)[:, None]
    print(f"mask: {mask}")
    mask_unshifted = jax.vmap(roll_row)(mask, -shift)
    print(f"mask_unshifted: {mask_unshifted}")

    def body_fun(i, arr):
        z = mask[:, i] * (gamma * arr[:, i+1] + rewards[:, i+1]) + (1-mask[:, i]) * arr[:, i]
        return arr.at[:, i].set(z)
    
    returns = jax.lax.fori_loop(2, capacity+1, lambda i, x: body_fun(capacity-i, x), arr)
    print(f"shifted returns: {returns}")
    returns = jax.vmap(roll_row)(returns, -shift)

    return returns
    

In [14]:
assign_returns()

reward: [[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]
rewards: [[3 4 0 1 2]
 [2 3 4 0 1]
 [4 0 1 2 3]]
arr: [[ 0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0. 10.]
 [ 0.  0.  0.  0. 10.]]
mask: [[False  True  True  True  True]
 [False False False False  True]
 [False  True  True  True  True]]
mask_unshifted: [[ True  True  True False  True]
 [False  True False False False]
 [ True  True  True  True False]]
shifted returns: [[ 0. 13. 13. 12. 10.]
 [ 0.  0.  0.  0. 10.]
 [ 0. 16. 15. 13. 10.]]


Array([[13., 12., 10.,  0., 13.],
       [ 0., 10.,  0.,  0.,  0.],
       [16., 15., 13., 10.,  0.]], dtype=float32)